#### **Coding Challenge Week 3**
**DE: Web Scraping Pipeline**

**Author: Lalit Gupta**

In [6]:
# !pip install jovian
# !pip install BeautifulSoup4

In [15]:
import requests
import csv
import pandas as pd
import os
from bs4 import BeautifulSoup

In [16]:
# web_url = 'https://books.toscrape.com/catalogue/page-'
# webpage = requests.get(web_url)
# content = webpage.text

In [17]:
#save content as html file
# with open('webpage.html', 'w', encoding='utf-8') as f:
#     f.write(content)

#### **WebScrapper Class** 

Handles Everything from Scraping, Parsing to Preprocessing of webpage data

In [25]:
class WebBookScraper():

    def __init__(self, url, limit=1):
        """
        Initializes the object with a URL and an optional limit.

        Parameters:
        - url: The URL to fetch data from.
        - limit: The limit on the number of items to fetch (default is 1).
        """
        self.url = url
        self.limit = limit

        # Create an empty DataFrame to store data
        self.df = pd.DataFrame(columns=['Title', 'Price', 'Availability','Rating'])
        self.df.to_csv('books.csv', index=False)
    
    def initiate(self):
        self.extract_info(self.url)

    def log(self, message):
        '''Prints a message to the console'''
        print("DEBUG: " + str(message))

    def transform_titles(self, titles):
        """
        Remove extra whitespace from book titles
        Remove commas from book titles
        Remove . from book titles
        """
        titles = [title.replace('\n','').replace(',','').replace('.','') for title in titles]
        return titles

    def get_book_titles(self, doc):
        """
        Extracts book titles from the given HTML document.

        Parameters:
        - doc: BeautifulSoup object representing the HTML document.

        Returns:
        - titles: List of book titles.
        """
        # title_tags = doc.find_all('h3 a[title]')
        books = doc.find_all("article", class_="product_pod")
        titles = [book.find("h3").find("a").get("title") for book in books]
        # titles = [tag.text for tag in title_tags]
        #remove the extra whitespace
        titles = self.transform_titles(titles)

        return titles

    def get_book_prices(self, doc):
        """
        Extracts book prices from the given HTML document.

        Parameters:
        - doc: BeautifulSoup object representing the HTML document.

        Returns:
        - prices: List of book prices.
        """
        price_tags = doc.find_all('p', class_='price_color')
        prices = [tag.text.replace('Â', '') for tag in price_tags]
        #convert prices to numeric values and remove the pound sign
        prices = [float(price[1:]) for price in prices]
        return prices
    
    def convert_rating(self, rating):
        """
        Converts rating from string to integer.

        Parameters:
        - rating: The string value for the rating.

        Returns:
        - rating: The integer value for the rating.
        """
        RATINGS = {
            'One': 1,
            'Two': 2,
            'Three': 3,
            'Four': 4,
            'Five': 5
        }
        return RATINGS[rating]

    def get_book_ratings(self, doc):
        """
        Extracts book ratings from the given HTML document.

        Parameters:
        - doc: BeautifulSoup object representing the HTML document.

        Returns:
        - ratings: List of book ratings.
        """
        rating_tags = doc.find_all('p', class_='star-rating')
        ratings = [tag.get('class')[1] for tag in rating_tags]
        #convert ratings to numeric values
        ratings = [self.convert_rating(rating) for rating in ratings]
        return ratings

    def get_stock_availability(self, doc):
        """
        Extracts stock availability information from the given HTML document.

        Parameters:
        - doc: BeautifulSoup object representing the HTML document.

        Returns:
        - availability: List of stock availability information.
        """
        stock_tags = doc.find_all('p', class_='instock availability')
        availability = [tag.text.strip() for tag in stock_tags]
        #If IN STOCK, then True, else False
        availability = [True if stock == 'In stock' else False for stock in availability]
        return availability

    def get_webpage(url):
        '''Get the webpage content'''
        webpage = requests.get(url)
        content = webpage.text
        soup = BeautifulSoup(content, 'html.parser')
        if webpage.status_code != 200:
            raise Exception('Failed to load page {}'.format(url))
        return soup
    
    def get_book_info(self, soup):
        '''Get the book info from the webpage'''
        titles,prices,availability,ratings = [],[],[],[]
        titles = self.get_book_titles(soup)
        prices = self.get_book_prices(soup)
        availability = self.get_stock_availability(soup)
        ratings = self.get_book_ratings(soup)
        return titles,prices,availability,ratings

    def extract_info(self, url):
        '''Pipeline for extracting data from the webpage'''
        for page in range(1,self.limit+1):
            self.log('Extracting page {} of {}'.format(page,self.limit))
            URL = url + str(page) + '.html'
            soup = WebBookScraper.get_webpage(URL)
            titles,prices,availability,ratings = self.get_book_info(soup)
            # create a dataframe from the extracted data
            df = pd.DataFrame({'Title':titles, 'Price':prices, 'Availability':availability, 'Rating':ratings})
            # append the dataframe to a csv file
            self.log('Saving books...')
            df.to_csv('books.csv', mode='a', header=False, index=False)
            self.log("Saved {} books from page {} of {}".format(len(titles),page,self.limit))
        self.log('Extraction complete!')


Initiating Web Scrapping ...

In [26]:
web_url = 'https://books.toscrape.com/catalogue/page-'
scraper = WebBookScraper(web_url,50)
scraper.initiate()

DEBUG: Extracting page 1 of 50
DEBUG: Saving books...
DEBUG: Saved 20 books from page 1 of 50
DEBUG: Extracting page 2 of 50
DEBUG: Saving books...
DEBUG: Saved 20 books from page 2 of 50
DEBUG: Extracting page 3 of 50
DEBUG: Saving books...
DEBUG: Saved 20 books from page 3 of 50
DEBUG: Extracting page 4 of 50
DEBUG: Saving books...
DEBUG: Saved 20 books from page 4 of 50
DEBUG: Extracting page 5 of 50
DEBUG: Saving books...
DEBUG: Saved 20 books from page 5 of 50
DEBUG: Extracting page 6 of 50
DEBUG: Saving books...
DEBUG: Saved 20 books from page 6 of 50
DEBUG: Extracting page 7 of 50
DEBUG: Saving books...
DEBUG: Saved 20 books from page 7 of 50
DEBUG: Extracting page 8 of 50
DEBUG: Saving books...
DEBUG: Saved 20 books from page 8 of 50
DEBUG: Extracting page 9 of 50
DEBUG: Saving books...
DEBUG: Saved 20 books from page 9 of 50
DEBUG: Extracting page 10 of 50
DEBUG: Saving books...
DEBUG: Saved 20 books from page 10 of 50
DEBUG: Extracting page 11 of 50
DEBUG: Saving books...
DEB

#### **Connecting to Snowflake**

In [53]:
# !pip install --upgrade snowflake-connector-python

In [27]:
import snowflake.connector
import os
USER='LALITBLEND360'
ACCOUNT='uypccvv-lnb97707'
PASSWORD = os.getenv('SNOWSQL_PWD')
conn = snowflake.connector.connect(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT
    )

**Data Warehouse & DataBase Configuration**

In [28]:
#Creating the warehouse and database
conn.cursor().execute("CREATE WAREHOUSE IF NOT EXISTS dw_blend360")
conn.cursor().execute("USE WAREHOUSE dw_blend360")

conn.cursor().execute("CREATE DATABASE IF NOT EXISTS books")
conn.cursor().execute("USE DATABASE books")


**Schema and Table**

In [29]:
conn.cursor().execute("CREATE SCHEMA IF NOT EXISTS public")
conn.cursor().execute("USE SCHEMA public")
# also create a table in the database

table_query = """
CREATE OR REPLACE TABLE table_books (
    Title VARCHAR,
    Price FLOAT,
    Availability BOOLEAN,
    Rating INTEGER
)
"""
conn.cursor().execute(table_query)

**Inserting data file to Snowflake**

In [30]:
# creating a stage
conn.cursor().execute("CREATE OR REPLACE STAGE books_stage")
# loading the data from the csv file to the stage
conn.cursor().execute("PUT file://books.csv @books_stage auto_compress=true")
# copying the data from the stage to the table
conn.cursor().execute("COPY INTO table_books FROM @books_stage/  FILE_FORMAT=(TYPE=CSV FIELD_DELIMITER=',' SKIP_HEADER=1)")

In [69]:
#select data from snowflake table
# conn.cursor().execute("SELECT * FROM table_books").fetchall()